In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm

from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

import os


#  클라이언트 설정



# 실행

In [21]:
folders = os.listdir('sql')
byshop_queries = []
shopby_queries = []

query_list = {
    'byshop' : byshop_queries,
    'shopby' : shopby_queries
}


for i, fd in enumerate(folders):
    files = os.listdir(f'sql/{folders[i]}')
    files = [f for f in files if f.endswith('.sql')]
    
    for file in files:
        with open(f'sql/{folders[i]}/{file}', 'r') as f:
            query =f.read()
            query_list[fd].append(query)



In [22]:
from datetime import datetime, date, timedelta
from google.cloud import bigquery
import time

def main_shopby(queries,historical_start_date,search_start_date):
    bq = bigquery.Client()
    
    
    # A: 데이터 셋의 시작 날짜 (고정)
    DATA_START_DATE = historical_start_date
    
    # B: 조회하려는 시작 날짜
    QUERY_START_DATE = search_start_date
    
    # C: 조회하려는 마감 날짜 (어제)
    QUERY_END_DATE = date.today() - timedelta(days=1)
    
    # 유효성 검사
    if QUERY_START_DATE < DATA_START_DATE:
        raise ValueError(f"조회 시작일({QUERY_START_DATE})은 데이터 시작일({DATA_START_DATE}) 이후여야 합니다.")
    if QUERY_END_DATE < QUERY_START_DATE:
        raise ValueError(f"조회 종료일({QUERY_END_DATE})은 조회 시작일({QUERY_START_DATE}) 이후여야 합니다.")
    
    # 날짜를 순차적으로 실행
    current_date = QUERY_START_DATE
    while current_date <= QUERY_END_DATE:
        cur_date = current_date.strftime('%Y-%m-%d')
        cur_date_plus = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
        
        print(f"\n=== {DATA_START_DATE.strftime('%Y-%m-%d')} 부터 {cur_date} 까지의 누적 데이터 처리 ===")
        print(f"기준일: {cur_date}, 다음날: {cur_date_plus}")

        # 각 쿼리를 개별적으로 실행
        for i, query in enumerate(queries, start=1):
            print(f"쿼리 {i} 실행 중...")
            try:
                formatted_query = query.format(
                    end_date=f"'{cur_date}'",
                    end_date_plus=f"'{cur_date_plus}'",
                    event_date_plus=f"'{cur_date}'"
                )
                
                job = bq.query(formatted_query)
                result = job.result()
                
                print(f"[쿼리 {i}] 성공")
                
            except Exception as e:
                print(f"[쿼리 {i}] 오류:", str(e))
                continue

        current_date += timedelta(days=1)

# query_list = {
#     'byshop' : byshop_queries,
#     'shopby' : shopby_queries
# }
tgt_query = "shopby"
search_start_date = date(2025, 1, 30) # 데이터 조회 날짜 기준.

if __name__ == "__main__":
    if tgt_query == "byshop":
        historical_start_date = date(2024, 3, 1)
    elif tgt_query == "shopby":
        historical_start_date = date(2024, 10, 1)
    else:
        historical_start_date = None
    total_start_time = time.time()
    
    main_shopby(query_list[tgt_query],historical_start_date,search_start_date)

    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    print(f"\n=== 전체 작업 완료 (총 소요시간: {total_duration:.2f}초) ===")



=== 2024-10-01 부터 2025-01-30 까지의 누적 데이터 처리 ===
기준일: 2025-01-30, 다음날: 2025-01-31
쿼리 1 실행 중...
[쿼리 1] 성공
쿼리 2 실행 중...
[쿼리 2] 성공
쿼리 3 실행 중...
[쿼리 3] 성공
쿼리 4 실행 중...
[쿼리 4] 성공
쿼리 5 실행 중...
[쿼리 5] 성공
쿼리 6 실행 중...
[쿼리 6] 성공
쿼리 7 실행 중...
[쿼리 7] 오류: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/ballosodeuk/queries/bfbfd2df-40de-4869-b08b-da75327967c7?maxResults=0&location=asia-northeast3&prettyPrint=false: Query error: Unrecognized name: member_no at [146:11]

Location: asia-northeast3
Job ID: bfbfd2df-40de-4869-b08b-da75327967c7


=== 전체 작업 완료 (총 소요시간: 39.98초) ===


In [ ]:
1/31 작업중

In [ ]:
"""CREATE OR REPLACE TABLE `ballosodeuk.ynam.rfm_shopby_history_array_table_tst` (
  user_id STRING,
  member_no STRING,
  -- 변경이 적은 기본 정보
  gender STRING,
  age_group STRING,
  join_group STRING,
  platform STRING,
  terms_agree_yn STRING,
  first_order_dt DATE,
  -- 스코어/상태 이력
  score_history ARRAY<STRUCT<
      snapshot_dt DATE,
      total_score FLOAT64,
      cut1 FLOAT64,
      cut2 FLOAT64,
      cut3 FLOAT64,
      grade STRING,
      risk_level STRING,
      
      cycle_length STRING,
      r_score INT64,
      f_score INT64,
      m_score INT64,
      term_score INT64,
      term_diff_score INT64,
      volatility_score INT64, -- 스키마와 일치하도록 INT64로 설정
      r_data INT64,
      f_data INT64,
      m_data INT64,
      ar_data FLOAT64,
  
      current_trailing_term FLOAT64,
      prev_trailing_term FLOAT64,
      term_diff FLOAT64,
      cycle_stddev FLOAT64,
      refund_rate FLOAT64,
      survival_prob FLOAT64,
      predicted_survival_time FLOAT64,
      pre_cash INT64,
      current_cash INT64,
      current_shoji INT64,
      earn INT64,
      spend INT64,
      exchange INT64,
      exchange_cash_rate FLOAT64,
      burnt INT64,
      total_accumulate_cash INT64,
      total_accumulate_shoji INT64
  >>,

  -- 카테고리 관심도 이력
  category_history ARRAY<STRUCT<
      snapshot_dt DATE,
      ranking_1_1 STRING,
      ranking_1_2 STRING,
      ranking_1_3 STRING,
      power_1_1 FLOAT64,
      power_1_2 FLOAT64,
      power_1_3 FLOAT64,
      ranking_2_1 STRING,
      ranking_2_2 STRING,
      ranking_2_3 STRING,
      ranking_2_4 STRING,
      ranking_2_5 STRING,
      ranking_2_6 STRING,
      power_2_1 FLOAT64,
      power_2_2 FLOAT64,
      power_2_3 FLOAT64,
      power_2_4 FLOAT64,
      power_2_5 FLOAT64,
      power_2_6 FLOAT64
  >>
);

MERGE `ballosodeuk.ynam.rfm_shopby_history_array_table_tst` T
USING (
    SELECT 
        user_id,
        member_no,
        gender,
        age_group,
        join_group,
        platform,
        terms_agree_yn,
        CAST(total_score AS FLOAT64) as total_score, 
        first_order_dt,
        refund_rate,
        r_score,
        f_score,
        m_score,
        cut1,
        cut2,
        cut3,
        grade,
        risk_level_shop as risk_level,
        current_trailing_term_shop as current_trailing_term,
        prev_trailing_term_shop as prev_trailing_term,
        term_diff_shop as term_diff,
        cycle_stddev_shop as cycle_stddev,
        ar_shop as ar_data,

        term_score,
        term_diff_score,
        volatility_score,
        cycle_length_shop AS cycle_length,
        r_shop AS r_data,
        f_shop AS f_data,
        m_shop AS m_data,
        suvival_prob_shop AS survival_prob, -- 오타 수정
        predicted_survival_time_shop AS predicted_survival_time,
        total_accumulate_cash,
        total_accumulate_shoji,
        pre_cash,
        current_cash,
        current_shoji,
        earn,
        spend,
        exchange,
        exchange_cash_rate,
        burnt,
        ranking_1_1,
        ranking_1_2,
        ranking_1_3,
        power_1_1,
        power_1_2,
        power_1_3,
        ranking_2_1,
        ranking_2_2,
        ranking_2_3,
        ranking_2_4,
        ranking_2_5,
        ranking_2_6,
        power_2_1,
        power_2_2, 
        power_2_3,
        power_2_4,
        power_2_5,
        power_2_6,
        CAST(snapshot_dt as DATE) as snapshot_dt
    FROM `ballosodeuk.ynam.rfm_table_shopby_rfm_target`
    WHERE user_id IS NOT NULL
) S
ON T.user_id = S.user_id
WHEN MATCHED THEN
    UPDATE
    SET
        member_no = S.member_no,
        gender = S.gender,
        age_group = S.age_group,
        join_group = S.join_group,
        platform = S.platform,
        terms_agree_yn = S.terms_agree_yn,
        first_order_dt = S.first_order_dt,

        score_history = ARRAY_CONCAT(
            IFNULL(T.score_history, []), 
            [
                CAST(
                    STRUCT(
                        S.snapshot_dt,
                        S.total_score,
                        S.cut1,
                        S.cut2,
                        S.cut3,
                        S.grade,
                        S.risk_level,

                        S.cycle_length,
                        S.r_score,
                        S.f_score,
                        S.m_score,
                        S.term_score,
                        S.term_diff_score,
                        S.volatility_score,
                        S.r_data,
                        S.f_data,
                        S.m_data,
                        S.ar_data,

                        S.current_trailing_term,
                        S.prev_trailing_term,
                        S.term_diff,
                        S.cycle_stddev,
                        S.refund_rate,
                        S.survival_prob,
                        S.predicted_survival_time,
                        S.pre_cash,
                        S.current_cash,
                        S.current_shoji,
                        S.earn,
                        S.spend,
                        S.exchange,
                        S.exchange_cash_rate,
                        S.burnt,
                        S.total_accumulate_cash,
                        S.total_accumulate_shoji
                    ) AS STRUCT<
                        snapshot_dt DATE,
                        total_score FLOAT64,
                        cut1 FLOAT64,
                        cut2 FLOAT64,
                        cut3 FLOAT64,
                        grade STRING,
                        risk_level STRING,

                        cycle_length STRING,
                        r_score INT64,
                        f_score INT64,
                        m_score INT64,
                        term_score INT64,
                        term_diff_score INT64,
                        volatility_score INT64,
                        r_data INT64,
                        f_data INT64,
                        m_data INT64,
                        ar_data FLOAT64,

                        current_trailing_term FLOAT64,
                        prev_trailing_term FLOAT64,
                        term_diff FLOAT64,
                        cycle_stddev FLOAT64,
                        refund_rate FLOAT64,
                        survival_prob FLOAT64,
                        predicted_survival_time FLOAT64,
                        pre_cash INT64,
                        current_cash INT64,
                        current_shoji INT64,
                        earn INT64,
                        spend INT64,
                        exchange INT64,
                        exchange_cash_rate FLOAT64,
                        burnt INT64,
                        total_accumulate_cash INT64,
                        total_accumulate_shoji INT64
                    >
                )
            ]
        ),

        category_history = ARRAY_CONCAT(
            IFNULL(T.category_history, []), 
            [
                CAST(
                    STRUCT(
                        S.snapshot_dt,
                        S.ranking_1_1,
                        S.ranking_1_2,
                        S.ranking_1_3,
                        S.power_1_1,
                        S.power_1_2,
                        S.power_1_3,
                        S.ranking_2_1,
                        S.ranking_2_2,
                        S.ranking_2_3,
                        S.ranking_2_4,
                        S.ranking_2_5,
                        S.ranking_2_6,
                        S.power_2_1,
                        S.power_2_2,
                        S.power_2_3,
                        S.power_2_4,
                        S.power_2_5,
                        S.power_2_6
                    ) AS STRUCT<
                        snapshot_dt DATE,
                        ranking_1_1 STRING,
                        ranking_1_2 STRING,
                        ranking_1_3 STRING,
                        power_1_1 FLOAT64,
                        power_1_2 FLOAT64,
                        power_1_3 FLOAT64,
                        ranking_2_1 STRING,
                        ranking_2_2 STRING,
                        ranking_2_3 STRING,
                        ranking_2_4 STRING,
                        ranking_2_5 STRING,
                        ranking_2_6 STRING,
                        power_2_1 FLOAT64,
                        power_2_2 FLOAT64,
                        power_2_3 FLOAT64,
                        power_2_4 FLOAT64,
                        power_2_5 FLOAT64,
                        power_2_6 FLOAT64
                    >
                )
            ]
        )
WHEN NOT MATCHED THEN
    INSERT (
        user_id, 
        member_no,
        gender,
        age_group,
        join_group,
        platform,
        terms_agree_yn,
        first_order_dt,
        score_history, 
        category_history
    )
    VALUES (
        S.user_id,
        S.member_no,
        S.gender,
        S.age_group,
        S.join_group,
        S.platform,
        S.terms_agree_yn,
        S.first_order_dt,
        [
            CAST(
                STRUCT(
                    S.snapshot_dt,
                    S.total_score,
                    S.cut1,
                    S.cut2,
                    S.cut3,
                    S.grade,    
                    S.risk_level,

                    S.cycle_length,
                    S.r_score,
                    S.f_score,
                    S.m_score,
                    S.term_score,
                    S.term_diff_score,
                    S.volatility_score,
                    S.r_data,
                    S.f_data,
                    S.m_data,
                    S.ar_data,

                    S.current_trailing_term,
                    S.prev_trailing_term,
                    S.term_diff,
                    S.cycle_stddev,
                    S.refund_rate,
                    S.survival_prob,
                    S.predicted_survival_time,
                    S.pre_cash,
                    S.current_cash,
                    S.current_shoji,
                    S.earn,
                    S.spend,
                    S.exchange,
                    S.exchange_cash_rate,
                    S.burnt,
                    S.total_accumulate_cash,
                    S.total_accumulate_shoji
                ) AS STRUCT<
                    snapshot_dt DATE,
                    total_score FLOAT64,
                    cut1 FLOAT64,
                    cut2 FLOAT64,
                    cut3 FLOAT64,
                    grade STRING,
                    risk_level STRING,

                    cycle_length STRING,
                    r_score INT64,
                    f_score INT64,
                    m_score INT64,
                    term_score INT64,
                    term_diff_score INT64,
                    volatility_score INT64,
                    r_data INT64,
                    f_data INT64,
                    m_data INT64,
                    ar_data FLOAT64,

                    current_trailing_term FLOAT64,
                    prev_trailing_term FLOAT64,
                    term_diff FLOAT64,
                    cycle_stddev FLOAT64,
                    refund_rate FLOAT64,
                    survival_prob FLOAT64,
                    predicted_survival_time FLOAT64,
                    pre_cash INT64,
                    current_cash INT64,
                    current_shoji INT64,
                    earn INT64,
                    spend INT64,
                    exchange INT64,
                    exchange_cash_rate FLOAT64,
                    burnt INT64,
                    total_accumulate_cash INT64,
                    total_accumulate_shoji INT64
                >
            )
        ],
        [
            CAST(
                STRUCT(
                    S.snapshot_dt,
                    S.ranking_1_1,
                    S.ranking_1_2,
                    S.ranking_1_3,
                    S.power_1_1,
                    S.power_1_2,
                    S.power_1_3,
                    S.ranking_2_1,
                    S.ranking_2_2,
                    S.ranking_2_3,
                    S.ranking_2_4,
                    S.ranking_2_5,
                    S.ranking_2_6,
                    S.power_2_1,
                    S.power_2_2,
                    S.power_2_3,
                    S.power_2_4,
                    S.power_2_5,
                    S.power_2_6
                ) AS STRUCT<
                    snapshot_dt DATE,
                    ranking_1_1 STRING,
                    ranking_1_2 STRING,
                    ranking_1_3 STRING,
                    power_1_1 FLOAT64,
                    power_1_2 FLOAT64,
                    power_1_3 FLOAT64,
                    ranking_2_1 STRING,
                    ranking_2_2 STRING,
                    ranking_2_3 STRING,
                    ranking_2_4 STRING,
                    ranking_2_5 STRING,
                    ranking_2_6 STRING,
                    power_2_1 FLOAT64,
                    power_2_2 FLOAT64,
                    power_2_3 FLOAT64,
                    power_2_4 FLOAT64,
                    power_2_5 FLOAT64,
                    power_2_6 FLOAT64
                >
            )
        ]
    )
"""